In [105]:
import pandas as pd
import numpy as np
import math
from treelib import Node, Tree
from functools import reduce
from IPython.display import Markdown, display

In [106]:
def calc_entropy(*attrs):
    np_attrs = np.array(attrs)
    if len(np_attrs[np_attrs > 0]) == 1:
        return 0
    elif np.all(np_attrs == attrs[0]):
        return 1

    total = np.sum(attrs)
    return reduce(
        lambda acc, x: acc + (-x / total) * math.log2(x / total),
        np_attrs[np_attrs > 0], 0.0)

In [28]:
def calc_average_info(parts_total, parts_entropy, total):
    parts = np.dstack((parts_total, parts_entropy))[0]
    return np.array(list(map(lambda item: (item[0] / total) * item[1], parts))).sum()

In [29]:
def calc_different(dataset, header, unique_amount):
    counted_frame = dataset[header].value_counts().values
    missing_items = unique_amount - len(counted_frame)
    return np.append(counted_frame, np.zeros(missing_items, dtype=np.int32)) if missing_items > 0 else counted_frame

In [111]:
def count_avarage_information(dataset, header, entire_dataset_total, unique_amount, show_log=True):
    grouped_by_attr = dataset.groupby(header)
    parts_total = []
    parts_entropy = []
    for key, item in grouped_by_attr:
        group = grouped_by_attr.get_group(key)
        different = calc_different(group, target_col_name, unique_amount)
        parts_total.append(np.sum(different))
        parts_entropy.append(calc_entropy(*different))

        if show_log:
            print("Different: ", different)
            display(group)
            print(end="\n\n")
    return calc_average_info(parts_total, parts_entropy, entire_dataset_total)

In [112]:
def build_decision_tree(dataset, unique_amount, parent=None, tree=Tree()):
    entire_different = calc_different(dataset, target_col_name, unique_amount)
    print("Entire different: ", entire_different)
    entire_dataset_total = np.sum(entire_different)
    entropy = calc_entropy(*entire_different)
    print("Entropy: ", entropy)

    average_info = pd.Series([0] * len(dataset.columns[:-1]), dtype=np.float32)
    average_info.index = dataset.columns[:-1]
    for header in dataset.columns[:-1]: 
        average_info[header] = count_avarage_information(dataset, header, entire_dataset_total, unique_amount)

    average_info = entropy - average_info
    max_impact = average_info.idxmax()
    tree_node_id = f"{max_impact}_{np.random.normal()}"
    tree.create_node(max_impact, tree_node_id, parent=parent)

    grouped_by_max_impact = dataset.groupby(max_impact)

    for key, item in grouped_by_max_impact:
        group = grouped_by_max_impact.get_group(key).copy()
        different = calc_different(group, target_col_name, unique_amount)
        group_entropy = calc_entropy(*different)

        group_value = group[max_impact].iloc[0]
        group_value_tree_id = f"{group_value}_{np.random.normal()}"
        tree.create_node(group_value, group_value_tree_id, parent=tree_node_id)
        if group_entropy == 0:
            tree.create_node(group[target_col_name].iloc[0], f"{group_value}_{np.random.normal()}", parent=group_value_tree_id)
        else:
            del group[max_impact]
            if len(group.columns) > 1:
                build_decision_tree(group, unique_values, parent=group_value_tree_id, tree=tree) 
            else:
                tree.create_node(group[target_col_name].iloc[0], f"{group_value}_{np.random.normal()}", parent=group_value_tree_id)
    return tree


In [113]:
credit_data = pd.read_csv("data.csv")
target_col_name = 'Риск'
credit_data = credit_data[[col for col in credit_data.columns if col != target_col_name] + [target_col_name]]
unique_values = len(credit_data[target_col_name].unique())

credit_data

,Кредитная_история,Долг,Поручительство,Доход,Риск
0,плохая,высокий,нет,От_0_до_15,высокий
1,неизвестная,высокий,нет,От_15_до_35,высокий
2,неизвестная,низкий,нет,От_15_до_35,средний
3,неизвестная,низкий,нет,От_0_до_15,высокий
4,неизвестная,низкий,нет,Больше_35,низкий
5,неизвестная,высокий,адекватное,Больше_35,низкий
6,плохая,низкий,нет,От_0_до_15,высокий
7,плохая,низкий,адекватное,Больше_35,средний
8,хорошая,низкий,нет,Больше_35,низкий
9,хорошая,высокий,адекватное,Больше_35,низкий


In [114]:
tree = build_decision_tree(credit_data, unique_values)

Entire different:  [6 5 3]
Entropy:  1.5306189948485172
Different:  [2 2 1]


,Кредитная_история,Долг,Поручительство,Доход,Риск
1,неизвестная,высокий,нет,От_15_до_35,высокий
2,неизвестная,низкий,нет,От_15_до_35,средний
3,неизвестная,низкий,нет,От_0_до_15,высокий
4,неизвестная,низкий,нет,Больше_35,низкий
5,неизвестная,высокий,адекватное,Больше_35,низкий




Different:  [3 1 0]


,Кредитная_история,Долг,Поручительство,Доход,Риск
0,плохая,высокий,нет,От_0_до_15,высокий
6,плохая,низкий,нет,От_0_до_15,высокий
7,плохая,низкий,адекватное,Больше_35,средний
13,плохая,высокий,нет,От_15_до_35,высокий




Different:  [3 1 1]


...

,Долг,Поручительство,Риск
1,высокий,нет,высокий




Different:  [1 0 0]


,Долг,Поручительство,Риск
2,низкий,нет,средний




Different:  [1 1 0]


,Долг,Поручительство,Риск
1,высокий,нет,высокий
2,низкий,нет,средний


In [37]:
tree.show()

Доход
├── Больше_35
│   └── Кредитная_история
│       ├── неизвестная
│       │   └── низкий
│       ├── плохая
│       │   └── средний
│       └── хорошая
│           └── низкий
├── От_0_до_15
│   └── высокий
└── От_15_до_35
    └── Кредитная_история
        ├── неизвестная
        │   └── Долг
        │       ├── высокий
        │       │   └── высокий
        │       └── низкий
        │           └── средний
        ├── плохая
        │   └── высокий
        └── хорошая
            └── средний



In [121]:
def one_r(dataset, show_log=True):
    value_columns = dataset.columns[:-1]
    unique_values = dataset[target_col_name].unique()
    total_records = len(dataset)

    min_error_frame = None
    min_error = math.inf
    min_class_name = ''
    for column_name in value_columns:
        if show_log:
            display(Markdown(f"**{column_name}**"))
        groups = dataset.groupby(column_name)
        frame = pd.DataFrame(columns=[*unique_values, "majority", "errors"])
        for idx, (key, item) in enumerate(groups):
            group = groups.get_group(key)[target_col_name].value_counts()
            major_ind = group.idxmax()
            total_errors = group.sum() - group[major_ind]
            group = group.append(pd.Series([major_ind, total_errors], index=['majority', 'errors']))
            group.name = key
            frame = frame.append(group)
        frame = frame.fillna(0)
        frame_error = frame['errors'].sum()
        frame_errors_relative = frame_error / total_records
        if min_error_frame is None or frame_errors_relative < min_error:
            min_error = frame_errors_relative
            min_class_name = column_name
            min_error_frame = frame.copy()

        if show_log:    
            display(frame)
            print(f"Total error: {frame_error} / {total_records} = {round(frame_errors_relative, 4)}")
            print(end="\n\n")
    return min_class_name, min_error, min_error_frame
 

In [122]:
min_class_name, min_error, min_error_frame = one_r(credit_data)
display(Markdown(f"**Major class {min_class_name} with error {round(min_error, 4)}**"))
display(min_error_frame)

**Кредитная_история**

,высокий,средний,низкий,majority,errors
неизвестная,2,1,2,высокий,3
плохая,3,1,0,высокий,1
хорошая,1,1,3,низкий,2


Total error: 6 / 14 = 0.4286




**Долг**

,высокий,средний,низкий,majority,errors
высокий,4,1,3,высокий,4
низкий,2,2,2,высокий,4


...

**Доход**

,высокий,средний,низкий,majority,errors
Больше_35,0,1,5,низкий,1
От_0_до_15,4,0,0,высокий,0
От_15_до_35,2,2,0,высокий,2


Total error: 3 / 14 = 0.2143




**Major class Доход with error 0.2143**

,высокий,средний,низкий,majority,errors
Больше_35,0,1,5,низкий,1
От_0_до_15,4,0,0,высокий,0
От_15_до_35,2,2,0,высокий,2
